Add your google drive to the colab notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Clone the github repo for the training the anomaly detection

In [ ]:
!git clone https://github.com/metric-space-ai/cflow_infrared_images.git
%cd cflow_infrared_images

Create the YAML file inputs

1. path to the images and mask

In [ ]:
path = "/content/drive/MyDrive/metric_space/IRBilder" #@param {type:"string"}
normal_dir = "normal" #@param {type:"string"}
abnormal_dir = "abnormal" #@param {type:"string"}
normal_test_dir = "normal_test" #@param {type:"string"}
mask = "/content/drive/MyDrive/metric_space/IRBilder/abnormal_test" #@param {type:"string"}

 2. Training hyperparameters

In [ ]:
split_ratio = 0.2 #@param {type:"number"}
image_size = 256 #@param {type:"integer"}
train_batch_size = 16 #@param {type:"integer"}
test_batch_size = 16 #@param {type:"integer"}
inference_batch_size = 16 #@param {type:"integer"}
fiber_batch_size = 64 #@param {type:"integer"}

Choose the model and corresponding configuration file

In [ ]:
MODEL = "cflow"
CONFIG_PATH = f"heat_anomaly/models/{MODEL}/ir_image.yaml"

Formatting the configuration file with the user input

In [ ]:
import yaml

with open(CONFIG_PATH, "r") as stream:
    try:
        config_entries = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

1. Adding the user-defined path to the YAML

In [ ]:
config_entries['dataset']['path'] = path
config_entries['dataset']['normal_dir'] = normal_dir
config_entries['dataset']['abnormal_dir'] = abnormal_dir
config_entries['dataset']['normal_test_dir'] = normal_test_dir
config_entries['dataset']['mask'] = mask

2. Adding the use defined hyperparameters

In [ ]:
config_entries['dataset']['split_ratio'] = split_ratio
config_entries['dataset']['image_size'] = image_size
config_entries['dataset']['train_batch_size'] = train_batch_size
config_entries['dataset']['test_batch_size'] = test_batch_size
config_entries['dataset']['inference_batch_size'] = inference_batch_size
config_entries['dataset']['fiber_batch_size'] = fiber_batch_size

3. Saving the revised configuration file

In [ ]:
with open(CONFIG_PATH, 'w') as yaml_file:
    yaml.dump(config_entries, yaml_file, default_flow_style=False)

Installing the required packages

In [ ]:
!pip install -r requirements.txt

Importing Libraries

In [ ]:
from pathlib import Path
from typing import Any, Dict

import os
import numpy as np
from PIL import Image
from pytorch_lightning import Trainer
from torchvision.transforms import ToPILImage

from heat_anomaly.config import get_configurable_parameters
from heat_anomaly.data import get_datamodule
from heat_anomaly.models import get_model
from heat_anomaly.pre_processing.transforms import Denormalize
from heat_anomaly.utils.callbacks import LoadModelCallback, get_callbacks

Prepare the model and load the configurations

In [ ]:
config = get_configurable_parameters(config_path=CONFIG_PATH)
model = get_model(config)
callbacks = get_callbacks(config)

Prepare the data for the training

In [ ]:
datamodule = get_datamodule(config)
datamodule.setup()
datamodule.prepare_data()

Training the detector

In [ ]:
trainer = Trainer(**config.trainer, callbacks=callbacks)
trainer.fit(model=model, datamodule=datamodule)

Validating the training

In [ ]:
load_model_callback = LoadModelCallback(weights_path=trainer.checkpoint_callback.best_model_path)
trainer.callbacks.insert(0, load_model_callback)
trainer.test(model=model, datamodule=datamodule)